In [ ]:
initial_investment = 1000000
yearly_sale_percentage = 0.04
capital_gains_tax = 0.26
dividend_tax = 0.26
monthly_sale_percentage = yearly_sale_percentage / 12

In [ ]:
import pandas as pd

def load_source_data(prices_file, bigmac_file):
    # Stock prices
    ts = pd.read_csv(prices_file)
    ts = ts.sort_values(by="timestamp")
    ts = ts.reset_index()
    ts = ts.drop(columns="index")
    df = pd.DataFrame({
    "timestamp": ts["timestamp"],
    "price": ts["adjusted close"],
    "dividend": ts["dividend amount"],
    })
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Big Mac index
    bigmac_pricedata = pd.read_csv(bigmac_file)
    country = "United States"
    bmprice = bigmac_pricedata[bigmac_pricedata.name == country][['date', 'dollar_price']]
    bmprice['date'] = pd.to_datetime(bmprice['date'])
    bmprice.rename(columns={"date": "timestamp", "dollar_price": "cpi"}, inplace=True)
    
    # Discard data that is not in the same time range for both datasets
    bmprice = bmprice[(bmprice['timestamp'] >= df['timestamp'].iloc[0]) & (bmprice['timestamp'] <= df['timestamp'].iloc[-1])]
    df = df[(df['timestamp'] >= bmprice['timestamp'].iloc[0]) & (df['timestamp'] <= bmprice['timestamp'].iloc[-1])]
    
    # Merge datasets
    df = pd.merge_asof(df, bmprice, on="timestamp")
    # Dividend per dollar
    df['dividend'] = df['dividend'] / df['price']
    # Normalize CPI to 1 and stock price to 100
    df['cpi'] = df['cpi'].fillna(bmprice['cpi'].iloc[0])
    df['cpi'] = df['cpi'] / df['cpi'].iloc[0]
    df['price'] = (df['price'] / df['price'].iloc[0] * 100).round(3)
    # Dividend per share
    df['dividend'] = (df['dividend'] * df['price']).round(3)
    return df

In [ ]:
src = load_source_data("monthly_adjusted_IBM.csv", "bigmac.csv")
src

In [ ]:
df = src.copy()
initial_shares = initial_investment / df['price'].iloc[0]
initial_price = df['price'].iloc[0]
initial_price, initial_shares
df['shares'] = initial_shares
df['perc. sold'] = monthly_sale_percentage
df['shares'] = (df['shares'] * (1 - df['perc. sold']).cumprod()).round(2)
df['shares sold'] = (df['shares'].shift(1).fillna(initial_shares) - df['shares']).round(2)
df['portfolio value'] = (df['shares'] * df['price']).round(2)
df['total port change'] = ((df['portfolio value'] / initial_investment)).round(4)
df['monthly value inc'] = (df['portfolio value'] - df['portfolio value'].shift(1).fillna(initial_investment)).round(2)
df['inflation adj. value'] = (df['portfolio value'] / df['cpi']).round(2)
df['infl. adj. port change'] = ((df['portfolio value'] / df['cpi'] / initial_investment)).round(4)
df['shares sold value'] = (df['shares sold'] * df['price']).round(2)
df['capital gains'] = ((df['price'] - initial_price) * df['shares sold']).round(2)
df['capital gains tax'] = (df['capital gains'] * capital_gains_tax).round(2)
df['tax %'] = (df['capital gains tax'] / df['shares sold value']).round(4)
df['net sale income'] = (df['shares sold'] * df['price'] - df['capital gains tax']).round(2)
df['dividend income'] = (df['shares'] * df['dividend']).round(2)
df['dividend tax'] = (df['dividend income'] * dividend_tax).round(2)
df['net dividend income'] = (df['dividend income'] - df['dividend tax']).round(2)
df['net income'] = (df['net sale income'] + df['net dividend income']).round(2)
df['infl. adj. total income'] = (df['net income'] / df['cpi']).round(2)
df['infl. adj. income change'] = ((df['infl. adj. total income'] / df['infl. adj. total income'].iloc[0])).round(4)
df

In [ ]:
dfs = [df[i:i+12] for i in range(0, df.shape[0], 12)]

In [ ]:
dfs = [df for df in dfs if df.shape[0] == 12]

In [ ]:
yearly = pd.DataFrame({
    "timestamp": [d["timestamp"].iloc[-1] for d in dfs],
    "price": [d["price"].iloc[-1] for d in dfs],
    "cpi": [d["cpi"].iloc[-1] for d in dfs],
    "shares": [d["shares"].iloc[-1] for d in dfs],
    "shares sold": [d["shares sold"].sum() for d in dfs],
    "portfolio value": [d["portfolio value"].iloc[-1] for d in dfs],
    "total port change": [d["total port change"].iloc[-1] for d in dfs],
    "inflation adj. value": [d["inflation adj. value"].iloc[-1] for d in dfs],
    "infl. adj. port change": [d["infl. adj. port change"].iloc[-1] for d in dfs],
    "capital gains tax": [d["capital gains tax"].sum() for d in dfs],
    "dividend tax": [d["dividend tax"].sum() for d in dfs],
    "net income": [d["net income"].sum() for d in dfs],
    "net sale income": [d["net sale income"].sum() for d in dfs],
    "net dividend income": [d["net dividend income"].sum() for d in dfs],
    "infl. adj. total income": [d["infl. adj. total income"].sum() for d in dfs],
    "infl. adj. monthly income": [d["infl. adj. total income"].mean() for d in dfs],
})  
yearly

In [ ]:
yearly['infl. adj. monthly income'] = yearly['infl. adj. monthly income'].round(2)
yearly['infl. adj. monthly income change'] = (yearly['infl. adj. monthly income'] / yearly['infl. adj. monthly income'].shift(1)).round(4)

In [ ]:
yearly

In [ ]:
yearly['infl. adj. monthly income total change'] = (yearly['infl. adj. monthly income'] / yearly['infl. adj. monthly income'].iloc[0]).round(4)

In [ ]:
yearly

In [ ]:
# plot portfolio value
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick

fig, ax = plt.subplots()
ax.plot(df['timestamp'], df['portfolio value'])
ax.set_title('Portfolio value')

In [ ]:
fig, ax = plt.subplots()
ax.plot(df['timestamp'], df['price'])
ax.set_title('Stock price')

In [ ]:
fig, ax = plt.subplots()
ax.plot(df['timestamp'], df['cpi'])
ax.set_title('Big Mac CPI')

In [ ]:
fig, ax = plt.subplots()
ax.plot(yearly['timestamp'], yearly['infl. adj. monthly income'])
ax.set_title('Inflation adjusted monthly income')